In [3]:
%matplotlib inline
import os
import matplotlib.pyplot as plt
import numpy as np
import sys
import pickle
import yaml
# import dcargs
import pandas as pd
import glob
from tqdm import tqdm
import matplotlib as mpl

# Setting default colormap settings
from palettable.scientific.diverging import Berlin_3
from palettable.lightbartlein.diverging import BlueOrange10_2, BlueOrange12_2, BlueOrangeRed_2
from palettable.matplotlib import Viridis_3
CMap = Viridis_3.get_mpl_colormap()

def get_colorMaps(nIds):
    weights = np.arange(0, nIds)

    norm = mpl.colors.Normalize(vmin=0, vmax=nIds)
    cmap = mpl.cm.ScalarMappable(norm=norm, cmap=CMap)

    return cmap

# path = "/Users/mohsen/Desktop/project/colab/with vito/new Results"
# path = "../results/hpc/central_network"
# path = "../results/hpc/spatial_network"
# path = "../results/hpc/spatial_network/individual_runs/envstd_0.0001_mnmax_0.0001_mnmin_0.0001_np1_0.2631578947368421_ow_0.0_a_n_100_run_0"
path = "../results/N100_2023-05-02-14-33-16_test_grid_search_Naive_centralized_random/results/envstd_7_mnint_0_mnmin_0_np1_0_ow_0_a_n_0_run_4"
sys.path.append(path)

# # path = "/home/mohsen/Project/colab/collective-decison-making-with-direl/results/2022-08-05-17-08-11_basic_experiment"

# path = "/home/mohsen/Project/colab/collective-decison-making-with-direl/results/2022-10-26-13-39-34_test_grid_search_naive"

In [4]:
# SHITTY INTROSPEKTIONEN

data_post_pro = parse_experiment_data_par(path)


# data_path = path + "/" +  "data.pickle5"

# with open(data_path, "rb") as f:
#     df_bayes = pickle.load(f)

NameError: name 'parse_experiment_data_par' is not defined

In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
import sys
import pickle
import yaml
import dcargs
import pandas as pd

from experiment import ExperimentParameters
import glob
from tqdm import tqdm
from yaml.loader import SafeLoader

from multiprocessing import Pool
import networkx as nx

from multiprocessing import Pool
import networkx as nx


# def parse_experiment_data_par(file_path):
file_path = path
if (True):
    # print("File Path is: " + file_path)
    # try:
    if(True):
        pickleFileName = file_path+"/data.pickle5"
        yamlFileName = file_path+"/config.yaml"

        with open(pickleFileName, "rb") as f:
            resFile = pickle.load(f)

        with open(yamlFileName, 'r') as f:
            configData = dcargs.from_yaml(ExperimentParameters, f)


        n_agents = configData.network_params.n_agents
        steps = configData.steps + 1
        z_gt = configData.true_value

        # not checked yet
        range_agent_measurement_noise = configData.network_params.max_agent_measurement_noise - configData.network_params.min_agent_measurement_noise
        mean_agent_measurement_noise = 0.5*(configData.network_params.max_agent_measurement_noise + configData.network_params.min_agent_measurement_noise)
        env_noise_std = configData.env_noise_std
        scalar_param1 = configData.network_params.scalar_param1
        weight_own_belief = configData.network_params.weight_own_belief

        colData = resFile['collective_data']
        adjc = colData['connectivity']
        agntData = colData['agent_data']

        # print("nAgents: " + str(n_agents) + ", steps: " + str(steps));
        # print("agnt data: ", len(agntData))

        zArr = np.zeros((n_agents,steps))
        pArr = zArr.copy()

        for i in range(n_agents):
            z = agntData[i].belief_mean
            std = agntData[i].belief_std
            zArr[i,:] = z
            pArr[i,:] = std

        avgZArr = np.mean(zArr, axis=0)
        E_t = (avgZArr-z_gt)**2
        E_p2 = (zArr - avgZArr)**2
        E_p = np.mean(E_p2, axis=0)
        E_a = E_t + E_p

        G = nx.from_numpy_array(adjc[0].cpu().detach().numpy())
        degreeDist = np.array(G.degree())[:, 1]
        clustCoef = np.array(list(nx.clustering(G).values()))
        eigVec = np.array(list(nx.eigenvector_centrality_numpy(G).values()))


        GG = nx.adjacency_matrix(G).toarray()
        row_sums = GG.sum(axis=1)
        row_sums[row_sums==0] = 1
        row_sums[np.isinf(row_sums)] = 1
        G2 = GG / row_sums[:, np.newaxis]
        eigVal = np.linalg.eigvals(G2)
        eigVal[np.isinf(eigVal)] = np.iinfo(np.int16).max
        # eigVal[np.isinf(eigVal) + np.isnan(eigVal)] = -1.01
        # eigVal = np.sort(np.abs(eigVal))

        # print("Start Testing Adjc Over Time! for steps: ", steps)
        eigvals_time = []
        # adjc_time = []
        # for t in range(steps):
        #     # print(t)
        #     G = nx.from_numpy_array(adjc[t].cpu().detach().numpy())
        #     GG = nx.adjacency_matrix(G).toarray()
        #     nx.draw(G)
        #     # plt.imshow(GG)
        #     # plt.show()
        #     row_sums = GG.sum(axis=1)
        #     row_sums[row_sums==0] = 1
        #     row_sums[np.isinf(row_sums)] = 1
        #     G2 = GG / row_sums[:, np.newaxis]
        #     # print("row sums: ",row_sums, flush=True)
        #     # print("/n")
        #     eigVal = np.linalg.eigvals(G2)
        #     # eigVal[np.isinf(eigVal) + np.isnan(eigVal)] = -1.01
        #     eigVal[np.isinf(eigVal)] = np.iinfo(np.int16).max
        #     # L = nx.normalized_laplacian_matrix(G)
        #     # eigVal = np.array(list(nx.eigenvector_centrality(G).values()))
        #     # print("eigVals lenght: ", eigVal.shape, flush=True)
        #     eigvals_time.append(eigVal)
        #     # adjc_time.append(adjc[t].cpu().detach().numpy())

        # print("eigVals list lenght: ", len(eigvals_time), flush=True)

        d = {"Trueness_Error": E_t, "Precision_Error": E_p, "Accuracy_Error": E_a, "Collective_Mean": avgZArr,
            "Number_of_Agents": [n_agents for _ in range(steps)], "True_Value": [z_gt for _ in range(steps)],
            "Timestep": [i for i in range(steps)],
            "mean_agent_measurement_noise": [mean_agent_measurement_noise for _ in range(steps)],
            "range_agent_measurement_noise": [range_agent_measurement_noise for _ in range(steps)],
            "std_environment_noise": [env_noise_std for _ in range(steps)],
            "scalar_param1": [scalar_param1 for _ in range(steps)],
            "weight_own_belief": [weight_own_belief for _ in range(steps)],
             # ToDo: Calculate Network properties for each time-step, so that we can see the evolution of these properties over time
            "Netw_num_of_Edges" : G.number_of_edges(),
            "Netw_std_degree" : np.std(degreeDist),
            "Netw_mean_degree" : np.mean(degreeDist),
            "Netw_std_eigVec" : np.std(eigVec),
            "Netw_mean_eigVec" : np.mean(eigVec),
            "Netw_max_eigVec" : np.max(eigVec),
            "Netw_std_CC" : np.std(clustCoef),
            "Netw_mean_CC" : np.mean(clustCoef),
            "Netw_max_CC" : np.max(clustCoef),
            "Netw_std_eigVal" : np.std(eigVal),
            "Netw_mean_eigVal" : np.mean(eigVal),
            "Netw_max_eigVal" : np.max(eigVal),#[-1],
            # "Netw_2ndmax_eigVal" : eigVal[-2],
            # "Netw_Adjc" : adjc_time,
            "Netw_eigVals_vs_time" : eigvals_time
            }
            # "Netw_eigVals_vs_time" : [get_eigVals_of_adj_mat(nx.from_numpy_array(adjc[i].cpu().detach().numpy())) for i in range(steps)]}

        # return pd.DataFrame.from_dict(d)
    # except Exception as e:
    else:
        print(e,flush=True)
        print("Something Went Wrong!",flush=True)
        print("File Path is: " + file_path,flush=True)

In [ ]:
pArr[:,10]

In [ ]:
plt.plot(zArr)
avgZArr = np.mean(zArr, axis=1)
upBnd = avgZArr + 3*np.std(zArr, axis=1)
loBnd = avgZArr - 3*np.std(zArr, axis=1)
plt.plot(avgZArr,"--b")
plt.plot(upBnd,"--r", loBnd, "--r",lw=0.5)
plt.xlabel("time step")
plt.ylabel("mean ind. belief")
plt.show()

In [ ]:
G.

Load Bayes DataFrame

In [ ]:

# experiment_name = "N25_2022-12-28-06-44-28_test_grid_search_Bayes"
# experiment_name = "N50_2022-12-28-13-53-37_test_grid_search_Bayes"
# experiment_name = "N100_2022-12-24-13-06-21_test_grid_search_Bayes"
# experiment_name = "N200_2022-12-27-17-25-46_test_grid_search_Bayes"


# with central network
experiment_name = "N100_2023-02-01-17-52-23_test_grid_search_Bayes_centralized_random"

# Spatial network
experiment_name = "N100_2023-02-03-12-16-56_test_grid_search_Bayes_spatial"

experiment_folder_path = path + "/" + experiment_name 

# dataFrame_path = path + "/" + experiment_name + "__df_withNet_overTime.pickle"
# dataFrame_path = path + "/" + experiment_name + "__df_withNet_overTime_WithAdjc.pickle"
# dataFrame_path = path + "/" + experiment_name + "__df_withNet_overTime_updatedEigs.pickle"
dataFrame_path = path + "/" + experiment_name + "__df_withNet_overTime_updatedEigs.pickle"


with open(dataFrame_path, "rb") as f:
    df_bayes = pickle.load(f)

Get the eigen values data from the DF

In [ ]:
df_bayes.groupby(["Timestep"])
# ,"Netw_eigVals_vs_time"]
mean_eig = []
std_eig = []
scnd_eig = []
time_list = []
scnd_eig_ratio = []
scnd_eig_t0 = 1
for i in range(len(df_bayes)): # range(10000):#
    eigVals = df_bayes.loc[i,'Netw_eigVals_vs_time']
    eigVals = np.sort(np.abs(eigVals))
    # print(eigVals[-2])
    # plt.plot(df_bayes.loc[i,'Timestep'], eigVals[-2],'.')
    # plt.plot(df_bayes.loc[i,'Timestep'], np.mean(eigVals),'.')
    time_list.append(df_bayes.loc[i,'Timestep'])
    mean_eig.append(np.mean(np.abs(eigVals)))
    std_eig.append(np.std(np.abs(eigVals)))
    scnd_eig_val = eigVals[-2]
    scnd_eig.append(scnd_eig_val)

    if(df_bayes.loc[i,'Timestep']==0):
        scnd_eig_t0 = scnd_eig_val
    scnd_eig_ratio.append(scnd_eig_val/scnd_eig_t0)

    if(df_bayes.loc[i,'Timestep']==10):
        time_list.append(np.nan)
        mean_eig.append(np.nan)
        std_eig.append(np.nan)
        scnd_eig.append(np.nan)
        scnd_eig_ratio.append(np.nan)

# plt.plot(np.array(time_list),np.array(mean_eig))
# plt.plot(np.array(time_list),np.array(scnd_eig),lw=0.1)
    # print(df_bayes.loc[i,'Timestep'])

Plot the eigen values over time

In [ ]:
plt.plot(np.array(time_list),np.array(mean_eig),lw=0.1)
plt.ylabel("mean eigs")
plt.show()
plt.plot(np.array(time_list),np.array(std_eig),lw=0.01)
plt.ylabel("std eigs")
plt.show()
plt.plot(np.array(time_list),np.abs(np.array(scnd_eig)),lw=0.1)
plt.ylabel("2nd eigs")
plt.show()
fig = plt.figure()
plt.plot(np.array(time_list),np.abs(np.array(scnd_eig_ratio)),lw=0.1)
plt.plot(np.array([0, 9]),np.array([1, 1]),lw=0.5,color='r')
plt.ylabel("2nd eigs/2nd eig @ t0")
fig.set_facecolor("w")
plt.show()

check any parameter of the DF

In [ ]:
# df_bayes.groupby(["Timestep"])
# # ,"Netw_eigVals_vs_time"]
# mean_eig = []
# std_eig = []
# scnd_eig = []
# time_list = []
# scnd_eig_ratio = []
# scnd_eig_t0 = 1
# for i in range(len(df_bayes)): # range(10000):#
#     eigVals = df_bayes.loc[i,'Netw_eigVals_vs_time']
#     eigVals = np.sort(np.abs(eigVals))
#     # print(eigVals[-2])
#     # plt.plot(df_bayes.loc[i,'Timestep'], eigVals[-2],'.')
#     # plt.plot(df_bayes.loc[i,'Timestep'], np.mean(eigVals),'.')
#     time_list.append(df_bayes.loc[i,'Timestep'])
#     mean_eig.append(np.mean(np.abs(eigVals)))
#     std_eig.append(np.std(np.abs(eigVals)))
#     scnd_eig_val = eigVals[-2]
#     scnd_eig.append(scnd_eig_val)

#     if(df_bayes.loc[i,'Timestep']==0):
#         scnd_eig_t0 = scnd_eig_val
#     scnd_eig_ratio.append(scnd_eig_val/scnd_eig_t0)

#     if(df_bayes.loc[i,'Timestep']==9):
#         time_list.append(np.nan)
#         mean_eig.append(np.nan)
#         std_eig.append(np.nan)
#         scnd_eig.append(np.nan)
#         scnd_eig_ratio.append(np.nan)

# # plt.plot(np.array(time_list),np.array(mean_eig))
# # plt.plot(np.array(time_list),np.array(scnd_eig),lw=0.1)
#     # print(df_bayes.loc[i,'Timestep'])

In [ ]:
# Checking a few math operations on network weights

import networkx as nx
G = nx.from_numpy_array(df_bayes.loc[50,"Netw_Adjc"])
# nx.draw(G)
GG = nx.adjacency_matrix(G).toarray()
row_sums = GG.sum(axis=1)
row_sums[row_sums==0] = 1
G2 = GG / row_sums[:, np.newaxis]
# print(GG.sum(axis=1))
# G2 = GG/GG.sum(axis=0)
# print(G2.sum(axis=1))
# print(np.sum(G2[1,:]))
# print(G2)
# print(G2[1,:])
# L = nx.normalized_laplacian_matrix(G)
# eigVal = np.linalg.eigvals(L.A)

eigVals = np.linalg.eigvals(G2)
print(eigVals)
plt.plot(np.sort(eigVals))


a = np.arange(0,27,3).reshape(3,3)
row_sums = a.sum(axis=1)
new_matrix = a / row_sums[:, np.newaxis]
print(a)
print(new_matrix)

In [ ]:
centrality = np.array(list(nx.eigenvector_centrality(G).values()))
print(centrality)
plt.plot(np.sort(centrality))

In [ ]:
eigs = df_bayes.loc[4500,'Netw_eigVals_vs_time']
plt.plot(np.sort(eigs))

In [ ]:
row_sums = np.random.uniform(0,1,10)
row_sums[4]=np.Inf
row_sums[row_sums==0] = 1
print(row_sums)
print(np.isinf(row_sums) + np.isnan(row_sums))

# eigVal[np.isinf(eigVal) + np.isnan(eigVal)] = -1
print(row_sums)

In [ ]:
eigs = df_bayes.loc[4500,'Netw_eigVals_vs_time']
plt.plot(np.sort(eigs))

Load Naive dataframe

In [ ]:
experiment_name_naive = "N25_2022-12-27-19-04-42_test_grid_search_Naive"
# experiment_name_naive = "N50_2022-12-23-17-32-03_test_grid_search_Naive"
# experiment_name_naive = "N100_2022-12-23-17-36-29_test_grid_search_Naive"
experiment_name_naive = "N200_2022-12-23-17-35-36_test_grid_search_Naive"

# Central networks
experiment_name_naive = "N100_2023-02-02-14-46-01_test_grid_search_Naive_centralized_random"

# Spatial Networks
experiment_name_naive = "N100_2023-02-03-12-15-22_test_grid_search_Naive_spatial"





experiment_folder_path_naive = path + "/" + experiment_name_naive 

dataFrame_path_naive = path + "/" + experiment_name_naive + "__df_withNet_updatedEigs.pickle"

with open(dataFrame_path_naive, "rb") as f:
    df_naive = pickle.load(f)

Filter the Bayes DF

In [ ]:
# mmu = 0.1
df_bayes_filtered = df_bayes; #df[df['mean agent measurement noise']==mmu]
# df_bayes_filtered = df[df['mean agent measurement noise']==mmu]
# time_index = 9
# df_bayes_filtered = df_bayes_filtered[df_bayes_filtered['Timestep']==time_index]
# n_agents = 100#32
# df_bayes_filtered = df_bayes_filtered[df_bayes_filtered['Number_of_Agents']==n_agents]
range_smallest = df_bayes_filtered['range_agent_measurement_noise'].min()
df_bayes_filtered = df_bayes_filtered[df_bayes_filtered['range_agent_measurement_noise']==range_smallest]
df_bayes_filtered = df_bayes_filtered[df_bayes_filtered['std_environment_noise']==df_bayes_filtered['mean_agent_measurement_noise']+range_smallest*0.5]


Filter the Naive DF

In [ ]:
# mmu = 0.1
df_naive_filtered = df_naive; #df[df['mean agent measurement noise']==mmu]
# df_filtered = df[df['mean agent measurement noise']==mmu]
time_index = 9
df_naive_filtered = df_naive_filtered[df_naive_filtered['Timestep']==time_index]
# df_naive_filtered = df_naive_filtered[df_naive_filtered['Number_of_Agents']==n_agents]
# df_naive_filtered = df_naive_filtered[df_naive_filtered['std environment noise']==df_naive_filtered['mean agent measurement noise']+range_smallest*0.5]


In [ ]:
print(df_naive_filtered["Collective_Mean"].max())
fig = plt.figure(1,figsize=(14,10))
fig.set_facecolor('w')

ax1 = plt.subplot(2,3,1); ax1.set_title("Trueness Error")
df_bayes_filtered.plot.hexbin(x="std environment noise", y="scalar param1", C="Trueness Error", reduce_C_function=np.mean, gridsize=15, ax=ax1);

ax5 = plt.subplot(2,3,2); ax5.set_title("Precision Error")
df_bayes_filtered.plot.hexbin(x="std environment noise",    y="scalar param1", C="Precision Error", reduce_C_function=np.mean, gridsize=15, ax=ax5);

ax8 = plt.subplot(2,3,3); ax8.set_title("Accuracy Error")
df_bayes_filtered.plot.hexbin(x="std environment noise",    y="scalar param1", C="Accuracy Error", reduce_C_function=np.mean, gridsize=15, ax=ax8);


ax1 = plt.subplot(2,3,4); ax1.set_title("Trueness Error")
df_naive_filtered.plot.hexbin(x="std environment noise", y="scalar param1", C="Trueness Error", reduce_C_function=np.mean, gridsize=15, ax=ax1);

ax5 = plt.subplot(2,3,5); ax5.set_title("Precision Error")
df_naive_filtered.plot.hexbin(x="std environment noise",    y="scalar param1", C="Precision Error", reduce_C_function=np.mean, gridsize=15, ax=ax5);

ax8 = plt.subplot(2,3,6); ax8.set_title("Accuracy Error")
df_naive_filtered.plot.hexbin(x="std environment noise",    y="scalar param1", C="Accuracy Error", reduce_C_function=np.mean, gridsize=15, ax=ax8);

Take a specifc instance of dataframe by defining scalar param 1, std env noise, mean agent noise, num agents

In [ ]:
# val_to_plot_bayes = df_bayes['Trueness Error'].mean()
scalar_param_vals = np.unique(df_bayes["scalar_param1"])
# weight_own_belief_vals = np.unique(df_naive.weight_own_belief)

plt.rc('font', size=14)

fig, axs = plt.subplots(3)
fig.set_size_inches(9,9)
# str_to_plot = "Accuracy" # "Precision" # "Trueness" # 
str_to_plot = "Precision" # "Trueness" #  
# str_to_plot = "Trueness" #  
# scalar_param_to_check = scalar_param_vals[19]#1.0
for scalar_param_to_check in scalar_param_vals:#[10:11]:
    std_environment_noise_to_check = np.unique(df_bayes["std_environment_noise"])[-1] # 1.0#0.0001
    # num_agents_to_check = 200
    df_bayes_filtered = df_bayes
    df_bayes_filtered = df_bayes_filtered[df_bayes_filtered["scalar_param1"]==scalar_param_to_check]
    df_bayes_filtered = df_bayes_filtered[df_bayes_filtered["std_environment_noise"]==std_environment_noise_to_check]# and
    # df_bayes_filtered = df_bayes_filtered[df_bayes_filtered["mean agent measurement noise"]==std_environment_noise_to_check]
    # df_bayes_filtered = df_bayes_filtered[df_bayes_filtered["Number_of_Agents"]==num_agents_to_check]
    
    df_naive_filtered = df_naive
    df_naive_filtered = df_naive_filtered[df_naive_filtered["scalar_param1"]==scalar_param_to_check]
    df_naive_filtered = df_naive_filtered[df_naive_filtered["std_environment_noise"]==std_environment_noise_to_check]# and
    df_naive_filtered = df_naive_filtered[df_naive_filtered["mean_agent_measurement_noise"]==std_environment_noise_to_check]
    # df_naive_filtered = df_naive_filtered[df_naive_filtered["Number_of_Agents"]==num_agents_to_check]
    # df_naive_filtered = df_naive_filtered[df_naive_filtered["weight_own_belief"]==weight_own_belief_vals[0]]
    
    if(str_to_plot=="Trueness"):
        axs[0].plot(df_bayes_filtered.groupby("Timestep").mean()["Trueness_Error"],color=CMap(scalar_param_to_check),label="E_T (B)")
        axs[1].plot(df_naive_filtered.groupby("Timestep").mean()["Trueness_Error"],color=CMap(scalar_param_to_check),label="E_T (N)")
        difference = df_bayes_filtered.groupby("Timestep").mean()["Trueness_Error"]-df_naive_filtered.groupby("Timestep").mean()["Trueness_Error"]
        axs[2].plot(difference.to_numpy(),color=CMap(scalar_param_to_check),label="E_T (B-N)")

    elif(str_to_plot=="Precision"):
        axs[0].plot(df_bayes_filtered.groupby("Timestep").mean()["Precision_Error"],color=CMap(scalar_param_to_check),label="E_P (B)")
        axs[1].plot(df_naive_filtered.groupby("Timestep").mean()["Precision_Error"],color=CMap(scalar_param_to_check),label="E_P (N)") #,color="b"
        difference = df_bayes_filtered.groupby("Timestep").mean()["Precision_Error"]-df_naive_filtered.groupby("Timestep").mean()["Precision_Error"]
        axs[2].plot(difference.to_numpy(),color=CMap(scalar_param_to_check),label="E_P (B-N)")

    elif(str_to_plot=="Accuracy"):
        axs[0].plot(df_bayes_filtered.groupby("Timestep").mean()["Accuracy_Error"],color=CMap(scalar_param_to_check),label="E_A (B)")
        axs[1].plot(df_naive_filtered.groupby("Timestep").mean()["Accuracy_Error"],color=CMap(scalar_param_to_check),label="E_A (N)") #,color="b"
        difference = df_bayes_filtered.groupby("Timestep").mean()["Accuracy_Error"]-df_naive_filtered.groupby("Timestep").mean()["Accuracy_Error"]
        axs[2].plot(difference.to_numpy(),color=CMap(scalar_param_to_check),label="E_A (B-N)")



if(str_to_plot=="Trueness"):
    axs[0].set_ylabel("$E_T$ for Bayes")
    axs[1].set_ylabel("$E_T$ for Naiive")
    axs[2].set_ylabel("$E_T$ (B-N)")

elif(str_to_plot=="Precision"):
    axs[0].set_ylabel("$E_P$ for Bayes")
    axs[1].set_ylabel("$E_P$ for Naiive")
    axs[2].set_ylabel("$E_P$ (B-N)")

elif(str_to_plot=="Accuracy"):
    axs[0].set_ylabel("$E_A$ for Bayes")
    axs[1].set_ylabel("$E_A$ for Naiive")
    axs[2].set_ylabel("$E_A$ (B-N)")

axs[1].sharey(axs[0])
axs[2].set_xlabel("Time Step")

axs[0].set_title(str_to_plot + " Error")

axs[0].set_yscale('log') 
axs[1].set_yscale('log') 

fig.set_facecolor("w")
fig.savefig(path + str_to_plot+"_logy.png")


In [ ]:
df_bayes[df_bayes["Timestep"]==9].groupby("scalar_param1")["Trueness_Error"].mean().plot(color='b')
df_naive[df_naive["Timestep"]==9].groupby("scalar_param1")["Trueness_Error"].mean().plot(color='r')

# ax = plt.gca()
# ax.set_face_color("w")
plt.show()


df_bayes[df_bayes["Timestep"] == 9].groupby("scalar_param1")["Precision_Error"].mean().plot(color='b')
df_naive[df_naive["Timestep"] == 9].groupby("scalar_param1")["Precision_Error"].mean().plot(color='r')

plt.show()

df_bayes[df_bayes["Timestep"] == 9].groupby("scalar_param1")["Netw_num_of_Edges"].mean().plot(color='b')
df_naive[df_naive["Timestep"] == 9].groupby("scalar_param1")["Netw_num_of_Edges"].mean().plot(color='r')

In [ ]:
# val_to_plot_bayes = df_bayes['Trueness Error'].mean()
scalar_param_vals = np.unique(df_naive["scalar_param1"])
# weight_own_belief_vals = np.unique(df_naive.weight_own_belief)

plt.rc('font', size=14)

fig, axs = plt.subplots(3)
fig.set_size_inches(9,9)
str_to_plot = "Accuracy" # "Precision" # "Trueness" # 
# str_to_plot = "Precision" # "Trueness" #  
# str_to_plot = "Trueness" #  
# scalar_param_to_check = scalar_param_vals[19]#1.0
for scalar_param_to_check in scalar_param_vals:#[10:11]:
    df_naive_filtered = df_naive
    df_naive_filtered = df_naive_filtered[df_naive_filtered["scalar_param1"]==scalar_param_to_check]
    # df_naive_filtered = df_naive_filtered[df_naive_filtered["std_environment_noise"]==std_environment_noise_to_check]# and
    # df_naive_filtered = df_naive_filtered[df_naive_filtered["mean_agent_measurement_noise"]==std_environment_noise_to_check]
    # df_naive_filtered = df_naive_filtered[df_naive_filtered["Number_of_Agents"]==num_agents_to_check]
    # df_naive_filtered = df_naive_filtered[df_naive_filtered["weight_own_belief"]==weight_own_belief_vals[0]]
    
    if(str_to_plot=="Trueness"):
        # axs[0].plot(df_bayes_filtered.groupby("Timestep").mean()["Trueness_Error"],color=CMap(scalar_param_to_check),label="E_T (B)")
        axs[1].plot(df_naive_filtered.groupby("Timestep").mean()["Trueness_Error"],color=CMap(scalar_param_to_check),label="E_T (N)")
        # difference = df_bayes_filtered.groupby("Timestep").mean()["Trueness_Error"]-df_naive_filtered.groupby("Timestep").mean()["Trueness_Error"]
        # axs[2].plot(difference.to_numpy(),color=CMap(scalar_param_to_check),label="E_T (B-N)")

    elif(str_to_plot=="Precision"):
        # axs[0].plot(df_bayes_filtered.groupby("Timestep").mean()["Precision_Error"],color=CMap(scalar_param_to_check),label="E_P (B)")
        axs[1].plot(df_naive_filtered.groupby("Timestep").mean()["Precision_Error"],color=CMap(scalar_param_to_check),label="E_P (N)") #,color="b"
        # difference = df_bayes_filtered.groupby("Timestep").mean()["Precision_Error"]-df_naive_filtered.groupby("Timestep").mean()["Precision_Error"]
        # axs[2].plot(difference.to_numpy(),color=CMap(scalar_param_to_check),label="E_P (B-N)")

    elif(str_to_plot=="Accuracy"):
        # axs[0].plot(df_bayes_filtered.groupby("Timestep").mean()["Accuracy_Error"],color=CMap(scalar_param_to_check),label="E_A (B)")
        axs[1].plot(df_naive_filtered.groupby("Timestep").mean()["Accuracy_Error"],color=CMap(scalar_param_to_check),label="E_A (N)") #,color="b"
        # difference = df_bayes_filtered.groupby("Timestep").mean()["Accuracy_Error"]-df_naive_filtered.groupby("Timestep").mean()["Accuracy_Error"]
        # axs[2].plot(difference.to_numpy(),color=CMap(scalar_param_to_check),label="E_A (B-N)")



if(str_to_plot=="Trueness"):
    axs[0].set_ylabel("$E_T$ for Bayes")
    axs[1].set_ylabel("$E_T$ for Naiive")
    axs[2].set_ylabel("$E_T$ (B-N)")

elif(str_to_plot=="Precision"):
    axs[0].set_ylabel("$E_P$ for Bayes")
    axs[1].set_ylabel("$E_P$ for Naiive")
    axs[2].set_ylabel("$E_P$ (B-N)")

elif(str_to_plot=="Accuracy"):
    axs[0].set_ylabel("$E_A$ for Bayes")
    axs[1].set_ylabel("$E_A$ for Naiive")
    axs[2].set_ylabel("$E_A$ (B-N)")

axs[1].sharey(axs[0])
axs[2].set_xlabel("Time Step")

axs[0].set_title(str_to_plot + " Error")

axs[0].set_yscale('log') 
axs[1].set_yscale('log') 

fig.set_facecolor("w")
fig.savefig(path + str_to_plot+"_logy.png")


In [ ]:
np.unique(df_bayes["std environment noise"])[5]

In [ ]:
cmap = CMap

fig, ax = plt.subplots(figsize=(8,1.5))
col_map = plt.get_cmap(cmap)
mpl.colorbar.ColorbarBase(ax, cmap=col_map, orientation = 'horizontal')
ax.set_xlabel("Scalar param1",fontdict={'size':20})

for label in (ax.get_xticklabels() + ax.get_yticklabels()):
    label.set_fontsize(16)

fig.set_facecolor("w")
fig.savefig(path+"ColorMap.png",dpi=300, bbox_inches = "tight")
plt.show()


In [ ]:
plt.plot(difference.to_numpy())

In [ ]:
print(df_naive_filtered)

In [ ]:
# df_bayes.groupby("scalar param1")["scalar param1"]
# print(np.unique(df_bayes["scalar param1"]))
# print(np.unique(df_bayes["std environment noise"]))
print(np.unique(df_naive_filtered.weight_own_belief))
# print(np.unique(df_bayes["Number of Agents"]))

In [ ]:
df_bayes["mean agent measurement noise"].min()

In [ ]:
x = df_bayes[['Trueness Error', "Timestep"]].groupby("Timestep").mean()
# x = df_bayes[['Trueness Error', "Timestep"]].groupby("Timestep").std()
print(x)

y = df_naive[['Trueness Error', "Timestep"]].groupby("Timestep").mean()
# y = df_naive[['Trueness Error', "Timestep"]].groupby("Timestep").std()
print(y)

In [ ]:
# plt.plot(np.sort(df_bayes_filtered["std environment noise"]))
# plt.plot(np.sort(df_naive_filtered["std environment noise"]))

plt.hist(df_naive_filtered["scalar_param1"])
# plt.hist(df_bayes_filtered["std environment noise"])


The Manual version of visualization

In [ ]:
fig = plt.figure(1,figsize=(14,10))
fig.set_facecolor('w')

# ax1 = plt.subplot(2,3,1); ax1.set_title("Trueness Error")
df_bayes_filtered.plot.hexbin(x="std environment noise", y="scalar param1",C="Trueness Error", reduce_C_function=np.mean, gridsize=15);#, ax=ax1);

# ax5 = plt.subplot(2,3,2); ax5.set_title("Precision Error")
# df_bayes_filtered.plot.hexbin(x="std environment noise",    y="scalar param1",C="Precision Error", reduce_C_function=np.mean, gridsize=15, ax=ax5);

# ax8 = plt.subplot(2,3,3); ax8.set_title("Accuracy Error")
# df_bayes_filtered.plot.hexbin(x="std environment noise",    y="scalar param1",C="Accuracy Error", reduce_C_function=np.mean, gridsize=15, ax=ax8);


# ax1 = plt.subplot(2,3,4); ax1.set_title("Trueness Error")
# df_naive_filtered.plot.hexbin(x="std environment noise", y="scalar param1",C="Trueness Error", reduce_C_function=np.mean, gridsize=15, ax=ax1);

# ax5 = plt.subplot(2,3,5); ax5.set_title("Precision Error")
# df_naive_filtered.plot.hexbin(x="std environment noise",    y="scalar param1",C="Precision Error", reduce_C_function=np.mean, gridsize=15, ax=ax5);

# ax8 = plt.subplot(2,3,6); ax8.set_title("Accuracy Error")
# df_naive_filtered.plot.hexbin(x="std environment noise",    y="scalar param1",C="Accuracy Error", reduce_C_function=np.mean, gridsize=15, ax=ax8);

In [ ]:
fig = plt.figure(1,figsize=(15,14))
fig.set_facecolor('w')
# plt.rcParams['figure.figsize'] = [10, 5]
ax1 = plt.subplot(3,3,1); ax1.set_title("Trueness Error")
df_filtered.plot.hexbin(x="mean agent measurement noise",   y="scalar param1",C="Trueness Error", reduce_C_function=np.mean, gridsize=15, ax=ax1);
ax2 = plt.subplot(3,3,2); ax2.set_title("Trueness Error")
df_filtered.plot.hexbin(x="std environment noise",    y="scalar param1",C="Trueness Error", reduce_C_function=np.mean, gridsize=15, ax=ax2);
ax3 = plt.subplot(3,3,3); ax3.set_title("Trueness Error")
df_filtered.plot.hexbin(x="mean agent measurement noise",   y="std environment noise", C="Trueness Error", reduce_C_function=np.mean, gridsize=15, ax=ax3);
# df_filtered.plot.hexbin(x="scalar param1",                  y="scalar param1",C="Trueness Error",  gridsize=15, ax=ax3);

ax4 = plt.subplot(3,3,4); ax4.set_title("Precision Error")
df_filtered.plot.hexbin(x="mean agent measurement noise",   y="scalar param1",C="Precision Error", reduce_C_function=np.mean, gridsize=15, ax=ax4);
ax5 = plt.subplot(3,3,5); ax5.set_title("Precision Error")
df_filtered.plot.hexbin(x="std environment noise",    y="scalar param1",C="Precision Error", reduce_C_function=np.mean, gridsize=15, ax=ax5);
ax6 = plt.subplot(3,3,6); ax6.set_title("Precision Error")
df_filtered.plot.hexbin(x="mean agent measurement noise",   y="std environment noise", C="Precision Error", reduce_C_function=np.mean, gridsize=15, ax=ax6);
# df_filtered.plot.hexbin(x="scalar param1",                  y="scalar param1",C="Precision Error", reduce_C_function=np.max, gridsize=15, ax=ax6);

ax7 = plt.subplot(3,3,7); ax7.set_title("Accuracy Error")
df_filtered.plot.hexbin(x="mean agent measurement noise",   y="scalar param1",C="Accuracy Error", reduce_C_function=np.mean, gridsize=15, ax=ax7);
ax8 = plt.subplot(3,3,8); ax8.set_title("Accuracy Error")
df_filtered.plot.hexbin(x="std environment noise",    y="scalar param1",C="Accuracy Error", reduce_C_function=np.mean, gridsize=15, ax=ax8);
ax9 = plt.subplot(3,3,9); ax8.set_title("Accuracy Error")
df_filtered.plot.hexbin(x="mean agent measurement noise",   y="std environment noise", C="Accuracy Error", reduce_C_function=np.mean, gridsize=15, ax=ax9);
# df_filtered.plot.hexbin(x="scalar param1",                  y="scalar param1",C="Accuracy Error", reduce_C_function=np.max, gridsize=15, ax=ax9);

# df_filtered.plot.hexbin(x="std environment noise", y="scalar param1", C="Precision Error", reduce_C_function=np.max, gridsize=15, ax=ax3);

In [ ]:

fig = plt.figure(1,figsize=(15,12))
# plt.rcParams['figure.figsize'] = [10, 5]
ax1 = plt.subplot(3,3,1)
df_filtered.plot.scatter(x="mean agent measurement noise",   y="scalar param1",c="Trueness Error",ax=ax1);
ax2 = plt.subplot(3,3,2)
df_filtered.plot.scatter(x="std environment noise",    y="scalar param1",c="Trueness Error",  ax=ax2);
ax3 = plt.subplot(3,3,3)
df_filtered.plot.scatter(x="mean agent measurement noise",   y="std environment noise", c="Trueness Error",   ax=ax3);
# df_filtered.plot.hexbin(x="scalar param1",                  y="scalar param1",C="Trueness Error",  ax=ax3);

ax4 = plt.subplot(3,3,4)
df_filtered.plot.scatter(x="mean agent measurement noise",   y="scalar param1",c="Precision Error",  ax=ax4);
ax5 = plt.subplot(3,3,5)
df_filtered.plot.scatter(x="std environment noise",    y="scalar param1",c="Precision Error",  ax=ax5);
ax6 = plt.subplot(3,3,6)
df_filtered.plot.scatter(x="mean agent measurement noise",   y="std environment noise", c="Precision Error",  ax=ax6);
# df_filtered.plot.hexbin(x="scalar param1",                  y="scalar param1",C="Precision Error",  ax=ax6);

ax7 = plt.subplot(3,3,7)
df_filtered.plot.scatter(x="mean agent measurement noise",   y="scalar param1",c="Accuracy Error",  ax=ax7);
ax8 = plt.subplot(3,3,8)
df_filtered.plot.scatter(x="std environment noise",    y="scalar param1",c="Accuracy Error",  ax=ax8);
ax9 = plt.subplot(3,3,9)
df_filtered.plot.scatter(x="mean agent measurement noise",   y="std environment noise", c="Accuracy Error",  ax=ax9);
# df_filtered.plot.hexbin(x="scalar param1",                  y="scalar param1",C="Accuracy Error",  ax=ax9);
